In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import nltk

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
comments = pd.read_csv('../input/sentiment-analysis-msa-phase-2/train.csv')

In [ ]:
comments.head()

In [ ]:
comments.shape

In [ ]:
comments.text.isnull().sum()

In [ ]:
comments = pd.read_csv('../input/sentiment-analysis-msa-phase-2/test.csv')

# TEXT PREPROCESS FUNCTION

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stopwords = stopwords.words('english')
porter_stemmer = PorterStemmer()
lemmatizer = nltk.WordNetLemmatizer()




def preprocessing(columntitle):
    comments.dropna(subset=['text'], inplace=True)
    columntitle = columntitle.str.lower()
    columntitle = columntitle.apply(lambda x: re.sub(r'[^\w\s]', '', str(x)) )
    columntitle = columntitle.apply(lambda x: ' '.join([word for word in nltk.word_tokenize(x) if word.lower() not in stopwords]))
    columntitle = columntitle.apply(lambda x: re.sub(r'\(?http\S+', '', str(x)))
    #columntitle = columntitle.apply(lambda x: ' '.join(nltk.word_tokenize(x)))
    lemmatized = [lemmatizer.lemmatize(word) for word in columntitle]
    columntitle = columntitle.apply(lambda x: ' '.join(
    [lemmatizer.lemmatize(word) for word in nltk.word_tokenize(x)]
        ) )
    return columntitle
    

In [ ]:
comments['text'] = preprocessing(comments['text'])

In [ ]:
comments.head()

In [ ]:
mapping = {'neutral': 0, 'negative': -1, 'positive': 1}

comments = comments.replace({'sentiment': mapping})

In [ ]:
mapping = {0: 'neutral', -1: 'negative', 1: 'positive'}

#comments['sentiment'].map(mapping) 
comments = comments.replace({'sentiment': mapping})

## Single step from tutorials

In [ ]:
#deal with null
comments.dropna(subset=['text'], inplace=True)
comments.text.isnull().sum()
#remove punctuation
comments.text = comments.text.apply(lambda x: re.sub(r'[^\w\s]', '', x) )
#remove stopwords
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
comments.text = comments.text.apply(lambda x: ' '.join([word for word in nltk.word_tokenize(x) if word.lower() not in stopwords]))
#Remove links
comments.text = comments.text.apply(lambda x: re.sub(r'\(?http\S+', '', str(x)))
#comments.text = comments.text.apply(lambda x: ' '.join(nltk.word_tokenize(x)))
# Stemming and Lemmatization
from nltk.stem import PorterStemmer
porter_stemmer = PorterStemmer()
sample = str(comments['text'])
text = nltk.word_tokenize(sample)
stemmed = [porter_stemmer.stem(word) for word in text]
#stemmed
lemmatizer = nltk.WordNetLemmatizer()
lemmatized = [lemmatizer.lemmatize(word) for word in text]
#lemmatized
comments.text = comments.text.apply(lambda x: ' '.join(
    [lemmatizer.lemmatize(word) for word in nltk.word_tokenize(x)]
        ) )

## Bigrams

In [ ]:
all_text = comments.iloc[:10000,:].text.str.cat(sep='. ')

In [ ]:
all_text = nltk.word_tokenize(all_text)

In [ ]:
def common_bigrams(tokenized_text, min_freq, top_n):
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    finder = BigramCollocationFinder.from_words(tokenized_text)
    finder.apply_freq_filter(min_freq) 
    finder.nbest(bigram_measures.pmi, top_n)
    return finder.nbest(bigram_measures.pmi, top_n)

print(common_bigrams(all_text, 10, 10))

## Wordcloud

In [ ]:
import wordcloud
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

sample = comments.iloc[:10000,:].text.str.cat(sep='. ')

In [ ]:
import random
def grey_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    return "hsl(0, 0%%, %d%%)" % random.randint(1, 20)

In [ ]:

stp = STOPWORDS.copy()
wc = WordCloud(background_color="green", max_words=50,  stopwords=stp)
# generate word cloud
wc.generate(sample)

plt.figure(figsize=(10,20))
plt.imshow(wc.recolor(color_func=grey_color_func, random_state=3))
plt.show()

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#FIRST, we initialize VADER so we can use it within our Python script
sid = SentimentIntensityAnalyzer()

In [ ]:
comments[['neg','neu','pos','compound']] = comments.text.apply(lambda x: pd.Series(sid.polarity_scores(x)))

## Training a naive bayes sentiment classifier

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn import metrics

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(comments.text)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, comments['sentiment'],test_size = 0.25, random_state = 5)

In [ ]:
nb = MultinomialNB()

#nb.fit(features_matrix, item_we_want_to_predict)
nb.fit(X_train, Y_train)

In [ ]:
predicted = nb.predict(X_test)
accuracy_score = metrics.accuracy_score(predicted, Y_test)

In [ ]:
print(str('{:04.2f}'.format(accuracy_score*100))+'%')

In [ ]:
from sklearn.naive_bayes import ComplementNB

#Fitting the model
CNB = ComplementNB()
CNB.fit(X_train, Y_train)

#evaluating the model
#from sklearn import metrics
accuracy_score = metrics.accuracy_score(CNB.predict(X_test),Y_test)

print(str('{:4.2f}'.format(accuracy_score*100))+'%')

In [ ]:
from sklearn.naive_bayes import BernoulliNB
BNB = BernoulliNB()
BNB.fit(X_train, Y_train)
accuracy_score_bnb = metrics.accuracy_score(BNB.predict(X_test),Y_test)
print('BNB accuracy = ' + str('{:4.2f}'.format(accuracy_score_bnb*100))+'%')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X2 = tfidf.fit_transform(comments.text)

GNB = GaussianNB()


#splitting the data in test and training
#from sklearn.model_selection() import train_test_split()
x_train, x_test, y_train, y_test = train_test_split(X2, comments['sentiment'],test_size=0.25,random_state=5)

#defining the model
#compilimg the model -> we are going to use already used models GNB, MNB, CNB, BNB
#fitting the model

CNB.fit(x_train, y_train)
accuracy_score_cnb = metrics.accuracy_score(CNB.predict(x_test), y_test)
print('accuracy_score_cnb = '+str('{:4.2f}'.format(accuracy_score_cnb*100))+'%')

GNB.fit(x_train.todense(), y_train)
accuracy_score_gnb = metrics.accuracy_score(GNB.predict(x_test.todense()), y_test)
print('accuracy_score_gnb = '+str('{:4.2f}'.format(accuracy_score_gnb*100))+'%')

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# To list:

sentences = comments['text'].tolist()
labels = comments['sentiment'].tolist()

In [ ]:
print(sentences[1])
print(labels[1])

In [ ]:
training_size = int(len(sentences) * 0.8)

training_sentences = sentences[0: training_size]
testing_senteces = sentences[: training_size]
training_labels = labels[0: training_size]
testing_labels = labels[: training_size]

# Put labels into list to use later:

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [ ]:
training_size

In [ ]:
vocab_size = 1000
embedding_dim = 16
max_length = 280
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_senteces)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(embedding_dim,
                         return_sequences=True)
))
model.add(tf.keras.layers.Dense(6, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
callbacks = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)
num_epochs=3
modelo = model.fit(training_padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final),
          callbacks=[callbacks])

In [ ]:
comments.sentiment.unique()

In [ ]:
sample2 = str(comments['text'])
words = nltk.word_tokenize(sample2)

In [ ]:
comments = comments.drop(columns=['textID']) 

In [ ]:
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}

In [ ]:
sentences = comments['text'].tolist()

In [ ]:
import tensorflow as tf

import tensorflow_datasets as tfds
import os

In [ ]:
comments.head()

In [ ]:
X_train = comments.loc[:13740,'text'].values
y_train = comments.loc[:13740,'sentiment'].values
X_test = comments.loc[13741:27480,'text'].values
y_test = comments.loc[13741:27480,'sentiment'].values

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer_obj = Tokenizer()
total_text = X_train + X_test
tokenizer_obj.fit_on_texts(total_text)

#pad sequences
max_length = max ([len(s.split()) for s in total_text])

#define vocabulary size
vocab_size = len(tokenizer_obj.word_index) + 1

X_train_tokens = tokenizer_obj.texts_to_sequences(X_train)
X_test_tokens = tokenizer_obj.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_tokens, maxlen = max_length, padding = 'post')
X_test_pad = pad_sequences(X_test_tokens, maxlen = max_length, padding = 'post')


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding

EMBEDDING_DIM = 128
lstm_out = 196

print('Build model...')

model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length=max_length))
model.add(LSTM(100))
#model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
#model.add(tf.keras.layers.Dense(6, activation='relu'))
#model.add(Dense(1,activation='sigmoid'))
model.add(Dense(2,activation='softmax'))

#model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
model.compile(loss = 'categorical_crossentropy', 
              optimizer='adam',
              metrics = ['accuracy'])

In [ ]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [ ]:
batch_size = 64
num_epochs = 5


model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), batch_size=batch_size, epochs=num_epochs)

## Train word2vec Embedding

In [ ]:
comments.to_csv('MSA_test.csv')